# Entity Explorer - Account
 <details>
     <summary>&nbsp;<u>Details...</u></summary>

 **Notebook Version:** 1.0<br>
 **Python Version:** Python 3.6 (including Python 3.6 - AzureML)<br>
 **Required Packages**: kqlmagic, msticpy, pandas, numpy, matplotlib, networkx, ipywidgets, ipython, dnspython, ipwhois, folium, maxminddb_geolite2<br>
 **Platforms Supported**:
 - Azure Notebooks Free Compute
 - Azure Notebooks DSVM
 - OS Independent

 **Data Sources Required**:
 - Log Analytics - SecurityAlert, SecurityEvent, HuntingBookmark, Syslog, AAD SigninLogs, AzureActivity, OfficeActivity, ThreatIndicator
 - (Optional) - VirusTotal, AlienVault OTX, IBM XForce, Open Page Rank, (all require accounts and API keys)
 </details>

 Brings together a series of queries and visualizations to help you determine the security state of an Account. The account can be a Windows or Linux account or an Azure Active Directory/Office 365 account.

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hunting-Hypothesis" data-toc-modified-id="Hunting-Hypothesis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hunting Hypothesis</a></span></li><li><span><a href="#Notebook-Initialization" data-toc-modified-id="Notebook-Initialization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Notebook Initialization</a></span><ul class="toc-item"><li><span><a href="#Get-Workspace-and-Authenticate" data-toc-modified-id="Get-Workspace-and-Authenticate-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Get Workspace and Authenticate</a></span></li></ul></li><li><span><a href="#Enter-account-name-and-query-time-window" data-toc-modified-id="Enter-account-name-and-query-time-window-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Enter account name and query time window</a></span></li><li><span><a href="#Data-Sources-available-to-query" data-toc-modified-id="Data-Sources-available-to-query-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Sources available to query</a></span></li><li><span><a href="#Search-for-Account-Name-in-Host,-AAD,-Azure-and-Office-Data" data-toc-modified-id="Search-for-Account-Name-in-Host,-AAD,-Azure-and-Office-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Search for Account Name in Host, AAD, Azure and Office Data</a></span><ul class="toc-item"><li><span><a href="#Query-Data-Sources" data-toc-modified-id="Query-Data-Sources-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Query Data Sources</a></span></li></ul></li><li><span><a href="#Display-logons-from-account-sources" data-toc-modified-id="Display-logons-from-account-sources-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Display logons from account sources</a></span></li><li><span><a href="#Related-Alerts-and-Hunting-Bookmarks" data-toc-modified-id="Related-Alerts-and-Hunting-Bookmarks-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Related Alerts and Hunting Bookmarks</a></span><ul class="toc-item"><li><span><a href="#Alerts" data-toc-modified-id="Alerts-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Alerts</a></span></li><li><span><a href="#Hunting/Investigation-Bookmarks" data-toc-modified-id="Hunting/Investigation-Bookmarks-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Hunting/Investigation Bookmarks</a></span></li></ul></li><li><span><a href="#Further-Investigation" data-toc-modified-id="Further-Investigation-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Further Investigation</a></span></li><li><span><a href="#Windows-Host" data-toc-modified-id="Windows-Host-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Windows Host</a></span><ul class="toc-item"><li><span><a href="#Host-Logon-Summary" data-toc-modified-id="Host-Logon-Summary-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Host Logon Summary</a></span></li><li><span><a href="#Threat-Intelligence-for-logon-IP-Addresses" data-toc-modified-id="Threat-Intelligence-for-logon-IP-Addresses-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Threat Intelligence for logon IP Addresses</a></span></li><li><span><a href="#Geolocation-and-ownership-for-source-logon-IP-addresses" data-toc-modified-id="Geolocation-and-ownership-for-source-logon-IP-addresses-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Geolocation and ownership for source logon IP addresses</a></span></li><li><span><a href="#Additional-Alerts-for-logged-on-hosts" data-toc-modified-id="Additional-Alerts-for-logged-on-hosts-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Additional Alerts for logged-on hosts</a></span><ul class="toc-item"><li><span><a href="#Additional-alerts-for-source-IP-addresses" data-toc-modified-id="Additional-alerts-for-source-IP-addresses-9.4.1"><span class="toc-item-num">9.4.1&nbsp;&nbsp;</span>Additional alerts for source IP addresses</a></span></li></ul></li><li><span><a href="#Additional-Investigation-Bookmarks-for-logged-on-hosts" data-toc-modified-id="Additional-Investigation-Bookmarks-for-logged-on-hosts-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>Additional Investigation Bookmarks for logged-on hosts</a></span></li></ul></li><li><span><a href="#Linux-Host" data-toc-modified-id="Linux-Host-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Linux Host</a></span><ul class="toc-item"><li><span><a href="#Host-Logon-Summary" data-toc-modified-id="Host-Logon-Summary-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Host Logon Summary</a></span></li><li><span><a href="#Threat-Intelligence-for-logon-IP-Addresses" data-toc-modified-id="Threat-Intelligence-for-logon-IP-Addresses-10.2"><span class="toc-item-num">10.2&nbsp;&nbsp;</span>Threat Intelligence for logon IP Addresses</a></span></li><li><span><a href="#Geolocation-and-ownership-for-source-logon-IP-addresses" data-toc-modified-id="Geolocation-and-ownership-for-source-logon-IP-addresses-10.3"><span class="toc-item-num">10.3&nbsp;&nbsp;</span>Geolocation and ownership for source logon IP addresses</a></span></li><li><span><a href="#Additional-Alerts-for-logged-on-hosts" data-toc-modified-id="Additional-Alerts-for-logged-on-hosts-10.4"><span class="toc-item-num">10.4&nbsp;&nbsp;</span>Additional Alerts for logged-on hosts</a></span><ul class="toc-item"><li><span><a href="#Additional-alerts-for-source-IP-addresses" data-toc-modified-id="Additional-alerts-for-source-IP-addresses-10.4.1"><span class="toc-item-num">10.4.1&nbsp;&nbsp;</span>Additional alerts for source IP addresses</a></span></li></ul></li><li><span><a href="#Additional-Investigation-Bookmarks-for-logged-on-hosts" data-toc-modified-id="Additional-Investigation-Bookmarks-for-logged-on-hosts-10.5"><span class="toc-item-num">10.5&nbsp;&nbsp;</span>Additional Investigation Bookmarks for logged-on hosts</a></span></li></ul></li><li><span><a href="#AAD/Office-Account" data-toc-modified-id="AAD/Office-Account-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>AAD/Office Account</a></span><ul class="toc-item"><li><span><a href="#Azure/Office-Summary" data-toc-modified-id="Azure/Office-Summary-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Azure/Office Summary</a></span></li><li><span><a href="#Threat-Intelligence-for-IP-Addresses" data-toc-modified-id="Threat-Intelligence-for-IP-Addresses-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Threat Intelligence for IP Addresses</a></span></li><li><span><a href="#Geolocation-and-ownership-for-source-IP-addresses" data-toc-modified-id="Geolocation-and-ownership-for-source-IP-addresses-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Geolocation and ownership for source IP addresses</a></span></li><li><span><a href="#Additional-alerts-for-source-IP-addresses" data-toc-modified-id="Additional-alerts-for-source-IP-addresses-11.4"><span class="toc-item-num">11.4&nbsp;&nbsp;</span>Additional alerts for source IP addresses</a></span></li></ul></li><li><span><a href="#Appendices" data-toc-modified-id="Appendices-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Appendices</a></span><ul class="toc-item"><li><span><a href="#Available-DataFrames" data-toc-modified-id="Available-DataFrames-12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>Available DataFrames</a></span></li><li><span><a href="#Saving-data-to-Excel" data-toc-modified-id="Saving-data-to-Excel-12.2"><span class="toc-item-num">12.2&nbsp;&nbsp;</span>Saving data to Excel</a></span></li></ul></li><li><span><a href="#Setup" data-toc-modified-id="Setup-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#msticpyconfig.yaml-configuration-File" data-toc-modified-id="msticpyconfig.yaml-configuration-File-13.1"><span class="toc-item-num">13.1&nbsp;&nbsp;</span><code>msticpyconfig.yaml</code> configuration File</a></span></li></ul></li></ul></div>

## Hunting Hypothesis
Our broad initial hunting hypothesis is that a we have received account name entity which is suspected to be compromised and is being used malicious manner in internal networks, we will need to hunt from a range of different positions to validate or disprove this hypothesis.

Before you start hunting please run the cells in [Setup](#Setup) at the bottom of this Notebook.

## Notebook Initialization
<details>
    <summary>&nbsp;<u>More details...</u></summary>
        If this is your first time running this Notebook please run the cells in in the <a href="#Setup">Setup</a> section before proceeding to ensure you have the required packages installed correctly. Similarly, if you see any import failures (ImportError) in the notebook, please make sure that you have read and run the cells <a href="#Setup">Setup</a> section first. This section is at the end of the notebook.
    </details>

In [ ]:
# Imports
import sys
import warnings

from msticpy.nbtools.utility import check_py_version

MIN_REQ_PYTHON = (3, 6)
check_py_version(MIN_REQ_PYTHON)

from IPython import get_ipython
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import pandas as pd

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_colwidth", 100)

from msticpy.data import QueryProvider
from msticpy.nbtools import *
from msticpy.sectools import *
from msticpy.nbtools.utility import md, md_warn
from msticpy.nbtools.wsconfig import WorkspaceConfig

WIDGET_DEFAULTS = {
    "layout": widgets.Layout(width="95%"),
    "style": {"description_width": "initial"},
}

# Some of our dependencies (networkx) still use deprecated Matplotlib
# APIs - we can't do anything about it so suppress them from view
from matplotlib import MatplotlibDeprecationWarning
warnings.simplefilter("ignore", category=MatplotlibDeprecationWarning)

ws_config = WorkspaceConfig()

### Get Workspace and Authenticate
<details>
    <summary>&nbsp;<u>Details...</u></summary>
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. 
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. 
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)
```

Note: you may occasionally see a JavaScript error displayed at the end of the authentication - you can safely ignore this.<br>
On successful authentication you should see a ```popup schema``` button.
To find your Workspace Id go to [Log Analytics](https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.
</details>

In [ ]:
# Authentication
qry_prov = QueryProvider(data_environment="LogAnalytics")
qry_prov.connect(connection_str=ws_config.code_connect_str)
table_index = qry_prov.schema_tables

## Enter account name and query time window
Type the account name that you want to search for and the time bounds over which you want to search. 

You can specify the account as:

- a simple user name (e.g. `alice`)
- a user principal name (`alice@contoso.com`)
- a qualified windows user name `mydomain\alice`

In the second two cases the domain qualifier will be stripped off before the search. The search is not case sensitive and will match full substrings. E.g. `bob` will match `domain\bob` and `bob@contoso.com` but not `bobg` or `bo`.

In [ ]:
accountname_text = widgets.Text(description='Enter the Account name to search for:', **WIDGET_DEFAULTS)
display(accountname_text)

In [ ]:
query_times = nbwidgets.QueryTime(units='day', max_before=200, before=5, max_after=7)
query_times.display()

In [ ]:
# Set up function to allow easy reference to common parameters for queries
def acct_query_params():
    return {
        "start": query_times.start,
        "end": query_times.end,
        "account_name": accountname_text.value,
    }

## Data Sources available to query
This shows all of the tables in the workspace with a string matching the account name entered.

In [ ]:
# KQL query for full text search of IP address and display all datatypes 
datasource_status = '''
search \'{account_name}\'
| where TimeGenerated >= datetime({start}) and TimeGenerated <= datetime({end})
| summarize RowCount=count() by Table=$table
'''.format(**acct_query_params())
%kql -query datasource_status
datasource_status_df = _kql_raw_result_.to_dataframe()

#Display result as transposed matrix of datatypes availabel to query for the query period 
if len(datasource_status_df) > 0:
    display(Markdown("###  <span style='color:blue'> "
                     + "Datasources available to query for Account "
                     + f"*{acct_query_params()['account_name']}* </span>"))
    display(datasource_status_df)
else:
    display(Markdown(f'### <span style="color:orange"> No datasources available to query for the query period </span>'))

## Search for Account Name in Host, Azure Active Directory (AAD), Azure and Office 365 Data.
### Query Data Sources

In [ ]:
# AAD
md("Searching for AAD activity...")
summarize_clause = """
| summarize arg_max(TimeGenerated, *) by UserPrincipalName, OperationName, 
  Identity, IPAddress, tostring(LocationDetails)
| project TimeGenerated, UserPrincipalName, Identity, IPAddress, LocationDetails"""

aad_signin_df = (qry_prov.Azure
                 .list_aad_signins_for_account(**acct_query_params(),
                                               add_query_items=summarize_clause)
                )

md("Searching for Azure activity...")
# Azure Activity
summarize_clause = """
| summarize arg_max(TimeGenerated, *) by Caller, OperationName, 
  CallerIpAddress, ResourceId
| project TimeGenerated, UserPrincipalName=Caller, IPAddress=CallerIpAddress"""

azure_activity_df = (qry_prov.Azure
                     .list_azure_activity_for_account(**acct_query_params(),
                                                      add_query_items=summarize_clause)
                    )

md("Searching for Office365 activity...")
# Office Activity
summarize_clause = """
| project TimeGenerated, UserId = tolower(UserId), OfficeWorkload, Operation, ClientIP, UserType
| summarize arg_max(TimeGenerated, *) by UserId, OfficeWorkload, ClientIP
| order by TimeGenerated desc"""

o365_activity_df = (qry_prov.Office365
                    .list_activity_for_account(**acct_query_params(),
                                               add_query_items=summarize_clause)
                    )

md("Searching for Windows logon activity...")
# Windows Host
summarize_clause = """
| extend LogonStatus = iff(EventID == 4624, "success", "failed")
| project TimeGenerated, TargetUserName, TargetDomainName, Computer, LogonType, SubjectUserName, 
  SubjectDomainName, TargetUserSid, EventID, IpAddress, LogonStatus 
| summarize arg_max(TimeGenerated, *) by TargetUserName, TargetDomainName, LogonType, Computer, LogonStatus"""

win_logon_df = (qry_prov.WindowsSecurity
                .list_logon_attempts_by_account(**acct_query_params(),
                                                add_query_items=summarize_clause)
               )

md("Searching for Linux logon activity...")
# Linux host
summarize_clause = """
| summarize arg_max(TimeGenerated, *) by LogonType, SourceIP, Computer, LogonResult"""

linux_logon_df = (qry_prov.LinuxSyslog
                  .list_logons_for_account(**acct_query_params(),
                                           add_query_items=summarize_clause)
                 )

rec_count = (
    len(aad_signin_df) + len(azure_activity_df) 
    + len(o365_activity_df) + len(win_logon_df) 
    + len(linux_logon_df)
)
md(f"Found {rec_count} records...")

## Display logons from account sources
**Choose Account to Explore**

In [ ]:
from collections import namedtuple
AccountDFs = namedtuple("AccountDFs", ["linux", "windows", "aad", "azure", "o365"])
account_dfs = AccountDFs(
    linux=linux_logon_df,
    windows=win_logon_df,
    aad=aad_signin_df,
    azure=azure_activity_df,
    o365=o365_activity_df,
)

# Combine into single data frame

lx_df = (linux_logon_df[["AccountName", "TimeGenerated"]]
        .groupby("AccountName")
        .max()
        .reset_index()
        .assign(Source="LinuxHostLogon"))

win_df = (win_logon_df[["TargetUserName", "TimeGenerated"]]
          .groupby("TargetUserName")
          .max()
          .reset_index()
          .rename(columns={"TargetUserName": "AccountName"})
          .assign(Source="WindowsHostLogon"))

o365_df = (o365_activity_df[["UserId", "TimeGenerated"]]
           .groupby("UserId")
           .max()
           .reset_index()
           .rename(columns={"UserId": "AccountName"})
           .assign(Source="O365Activity"))

aad_df = (aad_signin_df[["UserPrincipalName", "TimeGenerated"]]
          .groupby("UserPrincipalName")
          .max()
          .reset_index()
          .rename(columns={"UserPrincipalName": "AccountName"})
          .assign(Source="AADLogon"))

azure_df = (azure_activity_df[["UserPrincipalName", "TimeGenerated"]]
            .groupby("UserPrincipalName")
            .max()
            .reset_index()
            .rename(columns={"UserPrincipalName": "AccountName"})
            .assign(Source="AzureActivity"))


all_sources_df = pd.concat([lx_df, win_df, o365_df, aad_df, azure_df])


# Display the results that we've found
format_tuple = (lambda x: 
                (x.AccountName + "   " + x.Source
                 + " (Last activity: " + str(x.TimeGenerated) + ")",
                 x.AccountName + " " + x.Source))
accts_dict = {item[0]: item[1] for item in all_sources_df.apply(format_tuple, axis=1)}

def display_activity(selected_item):
    acct, source = selected_account(selected_item)
    utils.md(f"{acct} (source: {source})", "bold")
    if source == "LinuxHostLogon":
        display(linux_logon_df[linux_logon_df["AccountName"] == acct]
               .sort_values("TimeGenerated", ascending=True))
    if source == "WindowsHostLogon":
        display(win_logon_df[win_logon_df["TargetUserName"] == acct]
               .sort_values("TimeGenerated", ascending=True))
    if source == "AADLogon":
        display(aad_signin_df[aad_signin_df["UserPrincipalName"] == acct]
               .sort_values("TimeGenerated", ascending=True))
    if source == "AzureActivity":
        display(azure_activity_df[azure_activity_df["UserPrincipalName"] == acct]
               .sort_values("TimeGenerated", ascending=True))
    if source == "O365Activity":
        display(o365_activity_df[o365_activity_df["UserId"] == acct]
               .sort_values("TimeGenerated", ascending=True))

def selected_account(selected_acct):
    acct, source = selected_acct.value.split(" ")
    return acct, source

select_acct = nbwidgets.SelectString(
    item_dict=accts_dict,
    auto_display=True,
    description="Select an account to explore",
    #action=display_activity,
    height="200px",
    width="100%")

In [ ]:
display_activity(select_acct)

## Related Alerts and Hunting Bookmarks
### Alerts
Any alerts with a matching account name are shown here. Select an alert to view the contents.

In [ ]:
account_name, account_source = selected_account(select_acct)
related_alerts = qry_prov.SecurityAlert.list_related_alerts(
    **acct_query_params()
)

def print_related_alerts(alertDict, entityType, entityName):
    if len(alertDict) > 0:
        md(f"Found {len(alertDict)} different alert types related to this {entityType} (`{entityName}`)",
           "large, bold"
        )
        for (k, v) in alertDict.items():
            print(f"- {k}, # Alerts: {v}")
    else:
        md(f"No alerts for {entityType} entity `{entityName}`")


if isinstance(related_alerts, pd.DataFrame) and not related_alerts.empty:
    alert_items = (
        related_alerts[["AlertName", "TimeGenerated"]]
        .groupby("AlertName")
        .TimeGenerated.agg("count")
        .to_dict()
    )
    print_related_alerts(alert_items, "account", account_name)
    nbdisplay.display_timeline(
        data=related_alerts, title="Alerts", source_columns=["AlertName"], height=200
    )
else:
    display(Markdown("No related alerts found."))

def disp_full_alert(alert):
    global related_alert
    related_alert = SecurityAlert(alert)
    nbdisplay.display_alert(related_alert, show_entities=True)

if related_alerts is not None and not related_alerts.empty:
    related_alerts["CompromisedEntity"] = related_alerts["src_accountname"]
    display(Markdown("### Click on alert to view details."))
    rel_alert_select = nbwidgets.AlertSelector(
        alerts=related_alerts,
        action=disp_full_alert,
    )
    rel_alert_select.display()

### Hunting/Investigation Bookmarks
Any alerts with a matching account name are shown here. Select a bookmark to view the contents.

In [ ]:
acct_name = acct_query_params()["account_name"]
related_bkmark_df = qry_prov.AzureSentinel.list_bookmarks_for_entity(
    **acct_query_params(), entity_id=acct_name
)

def print_related_bkmk(bookmarks, entityType, entityName):
    if len(bookmarks) > 0:
        md(f"Found {len(bookmarks)} different bookmarks related to this {entityType} (`{entityName}`)",
           "large, bold"
        )
    else:
        md(f"No alerts for {entityType} entity `{entityName}`")


if isinstance(related_bkmark_df, pd.DataFrame) and not related_bkmark_df.empty:
    bookmarks = (related_bkmark_df
                 .apply(lambda x: (f"{x.BookmarkName} {x.Tags}  {x.TimeGenerated}", x.BookmarkId),
                        axis=1)
                 .tolist())
    print_related_bkmk(bookmarks, "account", account_name)
    nbdisplay.display_timeline(
        data=related_bkmark_df,
        title="Bookmarks",
        source_columns=["BookmarkName", "Tags"], height=200
    )
else:
    display(Markdown("No related bookmarks found."))

def disp_bookmark(bookmark_id):
    display(related_bkmark_df[related_bkmark_df["BookmarkId"] == bookmark_id].T)

if related_bkmark_df is not None and not related_bkmark_df.empty:
    display(Markdown("### Click on bookmark to view details."))
    rel_bkmk_select = nbwidgets.SelectString(
        item_list=bookmarks,
        action=disp_bookmark,
        auto_display=True
    )
    

## Further Investigation
Depending on the type of account (AAD or Host/Endpoint account) we can drill deeper to look at data specific to that account type.

In [ ]:
# Function definitions used below
# This cell should be executed before continuing further.

# WHOIS lookup function
from functools import lru_cache
from ipwhois import IPWhois
from ipaddress import ip_address

@lru_cache(maxsize=1024)
def get_whois_info(ip_lookup, show_progress=False):
    try:
        ip = ip_address(ip_lookup)
    except ValueError:
        return "Not an IP Address", {}
    if ip.is_private:
        return "private address", {}
    if not ip.is_global:
        return "other address", {}
    whois = IPWhois(ip)
    whois_result = whois.lookup_whois()
    if show_progress:
        print(".", end="")
    return whois_result["asn_description"], whois_result


ti_lookup = TILookup()
def check_ip_ti(df, ip_col):

    ip4_rgx = r"((?:[0-9]{1,3}\.){3}[0-9]{1,3})"
    src_ip_addrs = (df[[ip_col]]
                    .assign(IP_ext=lambda x: x[ip_col].str.extract(ip4_rgx, expand=False))
                    .drop(columns=ip_col)
                    .rename(columns={"IP_ext": ip_col})
                    .dropna()
                    .drop_duplicates()
                   )

    ti_results = ti_lookup.lookup_iocs(data=src_ip_addrs, obs_col=ip_col)
    ti_results = ti_results[ti_results["Severity"] > 0]

    ti_merged_df = df.merge(ti_results, how="left", left_on=ip_col, right_on="Ioc")
    return ti_results, ti_merged_df, src_ip_addrs


geo_lookup = GeoLiteLookup()
def check_geo_whois(ip_df, df, ip_col):
        
    geo_ips = geo_lookup.lookup_ip(ip_addr_list=list(ip_df[ip_col].values))
    # TODO replace
    ip_dicts = [{**ent.Location.properties, "IpAddress": ent.Address} for ent in geo_ips[1]]
    df_out = pd.DataFrame(data=ip_dicts)
    geo_df = df.merge(df_out, how="left", left_on=ip_col, right_on="IpAddress")

    geo_df[["ASNDesc", "WhoisResult"]] = (
        ip_df
        .apply(lambda x: get_whois_info(x[ip_col]),
               axis=1, result_type="expand"))

    return geo_df

# Based on the account type, advice the user where to go next.

acct, source = selected_account(select_acct)
md(f"Account '{acct}'. Source is '{source}'", "bold, large, blue")

goto = lambda x: display(Markdown(f"### For further analysis go to {x}"))
if source == "LinuxHostLogon":
    goto("go to [LinuxHostLogon](#Linux-Host)")
if source == "WindowsHostLogon":
    goto("go to [WindowsHostLogon](#Windows-Host)")
if source in ["AADLogon", "AzureActivity", "O365Activity"]:
    goto("go to [AAD/Office Account](#AAD/Office-Account)")

## Windows Host
For Windows accounts we look for the following types of data:

- Logon Summary
- Threat Intelligence reports for logon source IP Address(es)
- Geo location and Whois lookup for logon source IP Address(es)
- Additional alerts for the hosts where the account had logged on
- Additional bookmarks for the hosts where the account had logged on

In [ ]:
ext_logon_status = "| extend LogonStatus = iff(EventID == 4624, 'success', 'failed')"
all_win_logons = (qry_prov.WindowsSecurity
                  .list_logon_attempts_by_account(**acct_query_params(),
                                                 add_query_items=ext_logon_status))

### Host Logon Summary

In [ ]:
logon_summary = (all_win_logons
 .groupby("Computer")
 .agg(
     TotalLogons=pd.NamedAgg(column="EventID", aggfunc="count"),
     LogonResult=pd.NamedAgg(column="LogonStatus", aggfunc=lambda x: x.value_counts().to_dict()),
     IPAddresses=pd.NamedAgg(column="IpAddress", aggfunc=lambda x: x.unique().tolist()),
     LogonTypeCount=pd.NamedAgg(column="LogonType", aggfunc=lambda x: x.value_counts().to_dict()),
     FirstLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="min"),
     LastLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="max"),
  )
)

display(logon_summary)
nbdisplay.display_timeline(data=all_win_logons,
                           group_by="IpAddress",
                           source_columns=["Computer", "LogonStatus", "LogonType"],
                           title="Logons")

### Threat Intelligence for logon IP Addresses

In [ ]:
ti_results, all_win_logons_ti, src_ip_addrs_win = check_ip_ti(df=all_win_logons, ip_col="IpAddress")
if not ti_results.empty:
    md(f"{len(ti_results)} threat intelligence hits have been "
       + "matched on one or more source IP addresses.", "bold, red, large")
    md(" You should investigate these hosts using "
       + "the 'Entity Explorer - Windows Host' notebook", "bold, red" )
    md("Logon details for TI matches are in the `all_win_logons_ti` DataFrame")
    display(ti_results)
else:
    md("No additional items found for logged on hosts")

### Geolocation and ownership for source logon IP addresses

In [ ]:
all_win_logons_geo = check_geo_whois(src_ip_addrs_win, all_win_logons, "IpAddress")
md("Geolocations and ASN Owner for account logon source IP addresses. Information only", "bold")

(all_win_logons_geo[~all_win_logons_geo["CountryName"].isna()]
 .groupby(["Computer", "IpAddress", "CountryCode","CountryName", "City", "ASNDesc"])
 .agg(
     TotalLogons=pd.NamedAgg(column="EventID", aggfunc="count"),
     LogonResult=pd.NamedAgg(column="LogonStatus", aggfunc=lambda x: x.value_counts().to_dict()),
     LogonTypeCount=pd.NamedAgg(column="LogonType", aggfunc=lambda x: x.value_counts().to_dict()),
     FirstLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="min"),
     LastLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="max"),
  )
)


### Additional Alerts for logged-on hosts

In [ ]:
related_host_alerts = []
for host in all_win_logons["Computer"].unique():
    host_alerts = qry_prov.SecurityAlert.list_related_alerts(
        start=acct_query_params()["start"],
        end=acct_query_params()["end"],
        host_name=host
    )
    related_host_alerts.append(host_alerts)
    
related_host_alerts_df = pd.concat(related_host_alerts)

# Show host alerts that were not in the Account alerts list
related_host_alerts_df = related_host_alerts_df[~related_host_alerts_df["SystemAlertId"]
                                                .isin(related_alerts["SystemAlertId"])]
if not related_host_alerts_df.empty:
    md(f"{len(related_host_alerts_df)} additional alerts have been "
       + "triggered on one or more hosts.", "bold, red, large")
    md(" You should investigate these hosts using "
       + "the 'Entity Explorer - Windows Host' notebook", "bold, red" )
    display(related_host_alerts_df)
else:
    md("No additional alerts found")

#### Additional alerts for source IP addresses
We can also search for alerts that contain the IP addresses that were the origin of logons to the host.

In [ ]:
ip_list = ",".join(list(all_win_logons["IpAddress"].unique()))
related_ip_alerts_df = qry_prov.SecurityAlert.list_alerts_for_ip(
    start=acct_query_params()["start"],
    end=acct_query_params()["end"],
    source_ip_list=ip_list
)
# remove Account and host alerts already seen
related_ip_alerts_df = related_ip_alerts_df[~related_ip_alerts_df["SystemAlertId"]
                                            .isin(related_alerts["SystemAlertId"])]
related_ip_alerts_df = related_ip_alerts_df[~related_ip_alerts_df["SystemAlertId"]
                                            .isin(related_host_alerts_df["SystemAlertId"])]
if not related_ip_alerts_df.empty:
    md(f"{len(related_ip_alerts_df)} additional alerts have been "
       + "triggered from one or more source IPs.", "bold, red, large")
    md(" You should investigate these IPs using "
       + "the 'Entity Explorer - IP Address' notebook", "bold, red" )
    display(related_ip_alerts_df)
else:
    md("No additional alerts found.")

### Additional Investigation Bookmarks for logged-on hosts

In [ ]:
related_host_bkmks = []
for host in all_win_logons["Computer"].unique():
    host_bkmks = qry_prov.AzureSentinel.list_bookmarks_for_entity(
        start=acct_query_params()["start"],
        end=acct_query_params()["end"],
        entity_id=f"'{host}'"
    )
    related_host_bkmks.append(host_bkmks)
    
related_host_bkmks_df = pd.concat(related_host_bkmks)

# Show host bookmarks that were not in the Account bookmarks list
related_host_bkmks_df = related_host_bkmks_df[~related_host_bkmks_df["BookmarkId"]
                                              .isin(related_bkmark_df["BookmarkId"])]
if not related_host_bkmks_df.empty:
    md(f"{len(related_host_bkmks_df)} additional investigation bookmarks have been "
       + "found for one or more hosts.", "bold, red, large")
    md(" You should investigate these hosts using "
       + "the 'Entity Explorer - Windows Host' notebook", "bold, red" )
    display(related_host_bkmks_df)
else:
    md("No additional items found for logged on hosts")

## Linux Host
For Linux accounts we look for the following types of data:
- Logon Summary
- Threat Intelligence reports for logon source IP Address(es)
- Geo location and Whois lookup for logon source IP Address(es)
- Additional alerts for the hosts where the account had logged on
- Additional bookmarks for the hosts where the account had logged on


In [ ]:
all_lx_logons = (qry_prov.LinuxSyslog
                 .list_logons_for_account(**acct_query_params()))

### Host Logon Summary

In [ ]:
logon_summary = (all_lx_logons
 .groupby("Computer")
 .agg(
     TotalLogons=pd.NamedAgg(column="Computer", aggfunc="count"),
     FailedLogons=pd.NamedAgg(column="LogonResult", aggfunc=lambda x: x.value_counts().to_dict()),
     IPAddresses=pd.NamedAgg(column="SourceIP", aggfunc=lambda x: x.unique().tolist()),
     LogonTypeCount=pd.NamedAgg(column="LogonType", aggfunc=lambda x: x.value_counts().to_dict()),
     FirstLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="min"),
     LastLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="max"),
  )
)

display(logon_summary)
nbdisplay.display_timeline(data=all_lx_logons,
                           group_by="SourceIP",
                           source_columns=["Computer", "LogonResult", "LogonType"],
                           title="Logons");

### Threat Intelligence for logon IP Addresses

In [ ]:
ti_results_lx, all_lx_logons_ti, src_ip_addrs_lx = check_ip_ti(df=all_lx_logons, ip_col="SourceIP")

if not ti_results_lx.empty:
    md(f"{len(ti_results_lx)} threat intelligence hits have been "
       + "matched on one or more source IP addresses.", "bold, red, large")
    md(" You should investigate these hosts using "
       + "the 'Entity Explorer - Linux Host' notebook", "bold, red" )
    display(ti_results_lx)
else:
    md("No additional items found for logged on hosts")

### Geolocation and ownership for source logon IP addresses

In [ ]:
all_lx_logons_geo = check_geo_whois(src_ip_addrs_lx, all_lx_logons, "SourceIP")

md("Geolocations and ASN Owner for account logon source IP addresses. Information only", "bold")

(all_lx_logons_geo[~all_lx_logons_geo["CountryName"].isna()]
 .groupby(["Computer", "SourceIP", "CountryCode","CountryName", "City", "ASNDesc"])
 .agg(
     TotalLogons=pd.NamedAgg(column="SourceSystem", aggfunc="count"),
     LogonResult=pd.NamedAgg(column="LogonResult", aggfunc=lambda x: x.value_counts().to_dict()),
     LogonTypeCount=pd.NamedAgg(column="LogonType", aggfunc=lambda x: x.value_counts().to_dict()),
     FirstLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="min"),
     LastLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="max"),
  )
)

### Additional Alerts for logged-on hosts

In [ ]:
related_host_alerts = []
for host in all_lx_logons["Computer"].unique():
    host_alerts = qry_prov.SecurityAlert.list_related_alerts(
        start=acct_query_params()["start"],
        end=acct_query_params()["end"],
        host_name=host
    )
    related_host_alerts.append(host_alerts)
    
related_host_alerts_df = pd.concat(related_host_alerts)

# Show host alerts that were not in the Account alerts list
related_host_alerts_df = related_host_alerts_df[~related_host_alerts_df["SystemAlertId"]
                                                .isin(related_alerts["SystemAlertId"])]
if not related_host_alerts_df.empty:
    md(f"{len(related_host_alerts_df)} additional alerts have been "
       + "triggered on one or more hosts.", "bold, red, large")
    md(" You should investigate these hosts using "
       + "the 'Entity Explorer - Linux Host' notebook", "bold, red" )
    display(related_host_alerts_df[['TenantId','TimeGenerated','AlertDisplayName','ConfidenceLevel','ConfidenceScore','Computer','ExtendedProperties','Entities']])
else:
    md("No additional items found for logged on hosts")

#### Additional alerts for source IP addresses
We can also search for alerts that contain the IP addresses that were the origin of logons to the host.

In [ ]:
ip_list = ",".join(list(all_lx_logons["SourceIP"].unique()))
related_ip_alerts_df = qry_prov.SecurityAlert.list_alerts_for_ip(
    start=acct_query_params()["start"],
    end=acct_query_params()["end"],
    source_ip_list=ip_list
)
# remove Account and host alerts already seen
related_ip_alerts_df = related_ip_alerts_df[~related_ip_alerts_df["SystemAlertId"]
                                            .isin(related_alerts["SystemAlertId"])]
related_ip_alerts_df = related_ip_alerts_df[~related_ip_alerts_df["SystemAlertId"]
                                            .isin(related_host_alerts_df["SystemAlertId"])]
if not related_ip_alerts_df.empty:
    md(f"{len(related_ip_alerts_df)} additional alerts have been "
       + "triggered from one or more source IPs.", "bold, red, large")
    md(" You should investigate these IPs using "
       + "the 'Entity Explorer - IP Address' notebook", "bold, red" )
    display(related_ip_alerts_df)
else:
    md("No additional alerts found.")

### Additional Investigation Bookmarks for logged-on hosts

In [ ]:
related_host_bkmks = []
for host in all_lx_logons["Computer"].unique():
    host_bkmks = qry_prov.AzureSentinel.list_bookmarks_for_entity(
        start=acct_query_params()["start"],
        end=acct_query_params()["end"],
        entity_id=host
    )
    related_host_bkmks.append(host_bkmks)
    
related_host_bkmks_df = pd.concat(related_host_bkmks)

# Show host bookmarks that were not in the Account bookmarks list
related_host_bkmks_df = related_host_bkmks_df[~related_host_bkmks_df["BookmarkId"]
                                              .isin(related_bkmark_df["BookmarkId"])]
if not related_host_bkmks_df.empty:
    md(f"{len(related_host_bkmks_df)} additional investigation bookmarks have been "
       + "found for one or more hosts.", "bold, red, large")
    md(" You should investigate these hosts using "
       + "the 'Entity Explorer - Windows Host' notebook", "bold, red" )
    display(related_host_bkmks_df)
else:
    md("No additional items found for logged on hosts")

## AAD/Office Account
For an Azure Active Directory account we look for the following data:
- AAD Sign-on activity
- Azure Activity
- Office 365 operations
- Threat intelligence reports for the client IP Address used in any of these activities
- Geo location and Whois lookup for logon source IP Address(es)
- Additional alerts for the logon source IP Address(es)

In [ ]:
# Fetch the data
aad_sum_qry = """
| extend UserPrincipalName=tolower(UserPrincipalName)
| project-rename Operation=OperationName, AppResourceProvider=AppDisplayName"""
aad_signin_df = (qry_prov.Azure
                 .list_aad_signins_for_account(**acct_query_params(),
                                              add_query_items=aad_sum_qry)
                )

az_sum_qry = """
| extend UserPrincipalName=tolower(Caller)
| project-rename IPAddress=CallerIpAddress, Operation=OperationName,
  AppResourceProvider=ResourceProvider"""
azure_activity_df = (qry_prov.Azure
                     .list_azure_activity_for_account(**acct_query_params(),
                                                      add_query_items=az_sum_qry)
                    )

o365_sum_qry = """
| extend UserPrincipalName=tolower(UserId)
| project-rename IPAddress=ClientIP, ResourceId=OfficeObjectId,
  AppResourceProvider=OfficeWorkload"""
o365_activity_df = (qry_prov.Office365
                    .list_activity_for_account(**acct_query_params(),
                                               add_query_items=o365_sum_qry)
                    )


### Azure/Office Summary

In [ ]:
az_all_data = pd.concat([aad_signin_df, azure_activity_df, o365_activity_df], sort=False)
nbdisplay.display_timeline(data=az_all_data,
                          group_by="AppResourceProvider",
                          source_columns=["Operation", "IPAddress", "AppResourceProvider"],
                          title="Azure Operations by Provider")
nbdisplay.display_timeline(data=az_all_data,
                          group_by="IPAddress",
                          source_columns=["Operation", "IPAddress", "AppResourceProvider"],
                          title="Azure Operations by Source IP")
nbdisplay.display_timeline(data=az_all_data,
                          group_by="Operation",
                          source_columns=["Operation", "IPAddress", "AppResourceProvider"],
                          title="Azure Operations by Source IP");

In [ ]:
(az_all_data
.groupby(["UserPrincipalName", "Type", "IPAddress", "AppResourceProvider", "UserType"])
.agg(
     OperationCount=pd.NamedAgg(column="Type", aggfunc="count"),
     OperationTypes=pd.NamedAgg(column="Operation", aggfunc=lambda x: x.unique().tolist()),
     Resources=pd.NamedAgg(column="ResourceId", aggfunc="nunique"),
     FirstOperation=pd.NamedAgg(column="TimeGenerated", aggfunc="min"),
     LastOperation=pd.NamedAgg(column="TimeGenerated", aggfunc="max"),
  )
)

### Threat Intelligence for IP Addresses

In [ ]:
ti_results_az, all_az_ti, src_ip_addrs_az = check_ip_ti(df=az_all_data, ip_col="IPAddress")

if not ti_results_az.empty:
    md(f"{len(ti_results_az)} threat intelligence hits have been "
       + "matched on one or more source IP addresses.", "bold, red, large")
    md(" You should investigate these IP addresses using "
       + "the 'Entity Explorer - IP Address' notebook", "bold, red" )
    display(ti_results_az)
else:
    md("No additional items found")

### Geolocation and ownership for source IP addresses

In [ ]:
all_az_geo = check_geo_whois(src_ip_addrs_az, az_all_data, "IPAddress")

md("Geolocations and ASN Owner for source IP addresses. Information only", "bold")

(all_az_geo[~all_az_geo["CountryName"].isna()]
 .groupby(["UserPrincipalName", "IPAddress", "CountryCode","CountryName", "City", "ASNDesc"])
 .agg(
     TotalOperations=pd.NamedAgg(column="SourceSystem", aggfunc="count"),
     Operations=pd.NamedAgg(column="Operation", aggfunc=lambda x: x.value_counts().to_dict()),
     AppResources=pd.NamedAgg(column="AppResourceProvider", aggfunc=lambda x: x.unique().tolist()),
     FirstLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="min"),
     LastLogon=pd.NamedAgg(column="TimeGenerated", aggfunc="max"),
  )
)

### Additional alerts for source IP addresses

In [ ]:
ip_list = ",".join(list(src_ip_addrs_az["IPAddress"].unique()))
related_ip_alerts_df = qry_prov.SecurityAlert.list_alerts_for_ip(
    start=acct_query_params()["start"],
    end=acct_query_params()["end"],
    source_ip_list=ip_list
)
# remove Account and host alerts already seen
related_ip_alerts_df = related_ip_alerts_df[~related_ip_alerts_df["SystemAlertId"]
                                            .isin(related_alerts["SystemAlertId"])]
if not related_ip_alerts_df.empty:
    md(f"{len(related_ip_alerts_df)} additional alerts have been "
       + "triggered from one or more source IPs.", "bold, red, large")
    md(" You should investigate these IPs using "
       + "the 'Entity Explorer - IP Address' notebook", "bold, red" )
    display(related_ip_alerts_df)

## Appendices

### Available DataFrames

In [ ]:
print('List of current DataFrames in Notebook')
print('-' * 50)
current_vars = list(locals().keys())
for var_name in current_vars:
    if isinstance(locals()[var_name], pd.DataFrame) and not var_name.startswith('_'):
        print(var_name)

### Saving data to Excel
To save the contents of a pandas DataFrame to an Excel spreadsheet
use the following syntax
```
writer = pd.ExcelWriter('myWorksheet.xlsx')
my_data_frame.to_excel(writer,'Sheet1')
writer.save()
```

## Setup
If you have not run this Notebook before please run this cell before running the rest of the Notebook.

In [ ]:
import sys
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

# Package Installs - try to avoid if they are already installed
try:
    import Kqlmagic
    from ipwhois import IPWhois
    print('If you answer "n" this cell will exit with an error in order to avoid the pip install calls,')
    print('This error can safely be ignored.')
    resp = input('msticpy and Kqlmagic packages are already loaded. Do you want to re-install? (y/n)')
    if resp.strip().lower() != 'y':
        sys.exit('pip install aborted - you may skip this error and continue.')
    else:
        print('After installation has completed, restart the current kernel and run '
              'the notebook again skipping this cell.')
except ImportError:
    pass

print('\nPlease wait. Installing required packages. This may take a few minutes...')
!pip install msticpy --upgrade --user
!pip install ipwhois --upgrade --user
 
# Uncomment to refresh the maxminddb database
# !pip install maxminddb-geolite2 --upgrade 
print('To ensure that the latest versions of the installed libraries '
      'are used, please restart the current kernel and run '
      'the notebook again skipping this cell.')

### `msticpyconfig.yaml` configuration File
You can configure primary and secondary TI providers and any required parameters in the `msticpyconfig.yaml` file. This is read from the current directory or you can set an environment variable (`MSTICPYCONFIG`) pointing to its location.

Primary providers are used by default. Secondary providers can be invoked by using the `providers` parameter to lookup_ioc or lookup_iocs. `providers` should be a list of strings identifying the provider to use. The provider ID is given by the `Provider:` setting for each of the TI providers.

For most providers you will usually need to supply an authorization (API) key and in some cases a user ID for each provider.

For LogAnalytics/Azure Sentinel providers, you will need the workspace ID and tenant ID and will need to authenticate in order to access the data (although if you have an existing authenticated connection with the same workspace/tenant, this connection will be re-used). You can use a different workspace for TI from the one you are working in.

If you need to create a config file, uncomment the lines in the following cell.

**Warning** - this will overwrite a file of the same name in the current directory

Delete any provider entries that you do not want to use and add the missing parameters for your providers.

In [ ]:
# %%writefile msticpyconfig.yaml

# AzureSentinel:
#   #Workspaces:
#   #  Default:
#   #    WorkspaceId: "d973e3d2-28e6-458e-b2cf-d38876fb1ba4"
#   #    TenantId: "4cdf87a8-f0fc-40bb-9d85-68bcf4ac8e61"
#   #  Workspace2:
#   #    WorkspaceId: "c88dd3c2-d657-4eb3-b913-58d58d811a41"
#   #    TenantId: "4cdf87a8-f0fc-40bb-9d85-68bcf4ac8e61"
#   #  Workspace3:
#   #    WorkspaceId: "17e64332-19c9-472e-afd7-3629f299300c"
#   #    TenantId: "4ea41beb-4546-4fba-890b-55553ce6003a"
# QueryDefinitions:
#   # Uncomment and add paths to folders containing custom query definitions here
#   #Custom:
#   #  - /home/myuser/queries
# TIProviders:
#   OTX:
#     Args:
#       AuthKey: "your-otx-key"
#     Primary: True
#     Provider: "OTX" # Explicitly name provider to override
#   VirusTotal:
#     Args:
#       AuthKey: "your-vt-key"
#     Primary: True
#     Provider: "VirusTotal"
#   XForce:
#     # You can store items in an environment variable using this syntax
#     Args:
#       ApiID:
#         EnvironmentVar: "XFORCE_ID"
#       AuthKey:
#         EnvironmentVar: "XFORCE_KEY"
#     Primary: True
#     Provider: "XForce"
#   AzureSentinel:
#     Args:
#       # Workspace and tenant where your ThreatIndicator table is
#       # - usually the same as your default workspace
#       WorkspaceID: "c88dd3c2-d657-4eb3-b913-58d58d811a41"
#       TenantID: "4cdf87a8-f0fc-40bb-9d85-68bcf4ac8e61"
#     Primary: True
#     Provider: "AzSTI"